<a href="https://colab.research.google.com/github/AnnSenina/Python_for_CL/blob/main/homeworks/Hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Работа с файлами, pandas + парсинг
Собираем корпус эко-новостей с сайта новостного издания "Ведомости"

[По этой ссылке можно найти главную страницу](https://www.vedomosti.ru/ecology)

Что нам предстоит сделать:

достать все заголовки новостей в главной страницы + текст каждой новости

сохранить в датафрейм с колонками "ссылка", "дата", "заголовок", "текст"

сохранить датафрейм в файл

положить код и получившийся файл в свой репозиторий на гитхабе

### Способ первый, классический, как нас научили

In [1]:
import requests # Импортируем нужные библиотеки
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

In [2]:
url = 'https://www.vedomosti.ru/ecology/'
page = requests.get(url) # Запрашиваем страницу

In [3]:
page # Проверяем, что страница вернула код 200

<Response [200]>

In [4]:
soup = BeautifulSoup(page.text, 'html') # Парсим полученную страницу супом
#print(soup.prettify()) # Смотрим на ее структуру в красивом виде

In [5]:
# Найдем ссылки на выпуски новостей и создадим из них список
# Пути относительные, а нам надо абсолютные. Добавим к ссылкам нужную часть ссылки  
abs_urls = []
for link in soup.find_all('a'):
    if 'release' in link.get('href'):
        abs_url = 'https://www.vedomosti.ru' + link.get('href')
        if abs_url not in abs_urls: # Добавим условие, чтобы исключить повторное добавление ссылок
            abs_urls.append(abs_url)
abs_urls # Проверим, как заполнился список

['https://www.vedomosti.ru/ecology/release/2022/12/08',
 'https://www.vedomosti.ru/ecology/release/2022/11/10',
 'https://www.vedomosti.ru/ecology/release/2022/09/08',
 'https://www.vedomosti.ru/ecology/release/2022/07/21']

#### Вариант 1, используем функцию для обработки

In [19]:
# Эта функция достает из страницы нужную информацию:
# дату, заголовки, ссылки, если это страница выпуска, или же только текст, если это страница новости)

def EcoSpyder(url):
    
    issue_page = requests.get(url) # Запросили страницу
    issue_soup = BeautifulSoup(issue_page.text) # Обработаем каждую страницу выпуска новостей по ссылке супом

    # Проверим, что за страница, если выпуск, то вытащим дату, ссылки, заголовки

    if 'release' in url:
        dates, titles, links = [], [], []
        
        for d in issue_soup.find_all('time'):
            dates.append(d.text.strip())
        #dates = issue_soup.find('time').text  # Дата лежит в теге <time>. В выпуске все новости с одной датой, поэтому достаточно метода find.
        
        #date_raw = issue_soup.find('title').text # В качестве альтернативы дату можно достать регулярками из <title>
        #date = re.findall(r'[0-9]{2}\.[0-9]{2}\.[0-9]{4}', date_raw) 
    
        for k in issue_soup.find_all('a'): # Искомые заголовки и ссылки на сами тексты новостей находятся в тегах  <a>
            if k.get('data-vr-title') != None: # Название новости лежит в теге <data-vr-title>, поэтому из всех тегов <a> найдем те, где этот тег заполнен
                titles.append(k.get('data-vr-title')) 
                links.append('https://www.vedomosti.ru' + k.get('href')) # Сделаем ссылки абсолютными
                v = titles + links
                
        return dates, v #titles, links
   
    else: # На странице новости нам нужен только текст новости
        body = []
        
        for k in issue_soup.find_all('p', {'class': 'box-paragraph__text'}): # Текст лежит в тегах <p> с этим атрибутом 
            body.append(k.text)
            final_body = ' '.join(body) # Объединим строки в сплошной текст
        
        return final_body

In [20]:
#dates, titles, links, news_texts = [], [], [], [] # Создадим списки ключей и значений для словаря, из которого потом сделаем фрейм
keys, values = [], []

for i in range(len(abs_urls)): # Перебираем каждую ссылку на выпуск новостей
    newspiece = EcoSpyder(abs_urls[i]) # Вытаскиваем из нее нужную инфо
    
    #dates.append(newspiece[0])
    #titles.append(newspiece[1])
    #links.append(newspiece[2])
    print(newspiece)
    keys.append(newspiece[0]) # Распихиваем результат по спискам: дата - в ключи, она будет индексом
    values.append(newspiece[1:]) # Распихиваем результат по спискам: все остальное - в значения

print(keys)
print(*values)

#for l in range(len(values)): # Пройдем по словарю, по каждой дате выпуска 
#for m in range(len(values[1])): # Пройдем по каждой ссылке в каждом выпуске 
#    print(values[m])
#    news = EcoSpyder(values[m]) # Вытащим тексты
#    news_texts.append(news) # Добавим их в список

#news_dict = {keys[x]: list(values[x]) for x in range(len(keys))} # Скомпонуем словарь
#news_dict
        
#dates = [b for a in dates for b in a]
#titles = [b for a in titles for b in a]
#links = [b for a in links for b in a]
#print(*links) # Посмотрим, что на выходе.
#print(len(titles))
#print(len(news_texts))
#print(news_texts)

(['08.12.2022', '08.12.2022', '08.12.2022', '08.12.2022'], ['Энергетика с нулевым следом', '«Россия» станет самым мощным ледоколом в мире', 'Альтернатива «большому» атому', 'Атомная экология', 'https://www.vedomosti.ru/ecology/climate/articles/2022/12/08/954315-energetika-nulevim-sledom', 'https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/08/954309-rossiya-stanet-samim-moschnim-ledokolom-v-mire', 'https://www.vedomosti.ru/ecology/science_and_technology/articles/2022/12/08/954298-alternativa-bolshomu-atomu', 'https://www.vedomosti.ru/ecology/protection_nature/articles/2022/12/08/954286-atomnaya-ekologiya'])
(['10.11.2022', '10.11.2022', '10.11.2022', '10.11.2022'], ['Россия теплеет вдвое быстрее, чем остальной мир', 'Запретят ли собирать кедровые орехи', '«В России отсутствует специализированная инфраструктура сбора текстильных отходов»', '«Рубка байкальского леса приведет к непоправимым последствиям для экосистемы озера»', 'https://www.vedomosti.ru/ecology/climat

In [ ]:
news_dict = dict((z[0], list(z[1:])) for z in zip(dates, values))
#values = list(zip(links, titles, news_texts))
#for i in range(len(values)):
#    list(values[i])
#print(values)
print(news_dict)
print()

In [ ]:
news_dict = {zip(dates, values)}
news_dict

In [ ]:
print(*zip(list2, list3))

In [ ]:
dict(zip(list1, zip(list2, list3)))

In [ ]:
dict(zip(list1, zip(map(int, list2), map(int, list3))))

In [ ]:
dict(zip(list1, map(list, zip(map(int, list2), map(int, list3)))))

In [ ]:
dict(zip(list1, map(list, zip(*(map(int, lst) for lst in (list2, list3, list4))))))

news_df = pd.DataFrame.from_dict(news_dict, orient="columns").T.rename(columns={0:"city", 1:"age"})

In [ ]:


news_df = pd.DataFrame()
#news_df = pd.DataFrame(values, index=dates, columns=['title', 'link'])
display(news_df)

In [ ]:
df = pd.DataFrame.from_dict(news_dict).T
display(df)